## 1. Import libraries

In [ ]:
!pip install pyvi

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from pyvi import ViTokenizer

## 2. Read data

In [ ]:
# Đọc dữ liệu từ file json
data = pd.read_json("../input/mydata/data_coment_tiki.json")
data.head()

In [ ]:
data.rename(columns={"content":"text","rating":"star"}, inplace=True)
data.head()

In [ ]:
data.info()

### Do dữ liệu bị mất cân bằng nên ta xóa ngẫu nhiên 9000 mẫu 5 sao, và 1000 mẫu 4 sao

In [ ]:
np.random.seed(10)
remove_n = 10000
drop_indices = np.random.choice(data[data['star'] == 5].index, remove_n, replace=False)
data = data.drop(drop_indices)

In [ ]:
np.random.seed(10)
remove_n = 1000
drop_indices = np.random.choice(data[data['star'] == 4].index, remove_n, replace=False)
data = data.drop(drop_indices)

In [ ]:
len(data)

In [ ]:
data.star.value_counts()

## 3. Exploratory Data Analysis (EDA)

### Class distribution

In [ ]:
sns.histplot(data=data['star']);

### Number of characters in text

In [ ]:
def length_text(text):
    return len(text)

In [ ]:
data['length'] = data['text'].apply(length_text)
data['length'].head()

In [ ]:
plt.rcParams['figure.figsize'] = (18.0, 6.0)
bins = 150
plt.hist(data[data['star'] == 5]['length'], alpha = 0.6, bins=bins, label='5')
plt.hist(data[data['star'] == 4]['length'], alpha = 0.8, bins=bins, label='4')
plt.hist(data[data['star'] == 3]['length'], alpha = 0.8, bins=bins, label='3')
plt.hist(data[data['star'] == 2]['length'], alpha = 0.8, bins=bins, label='2')
plt.hist(data[data['star'] == 1]['length'], alpha = 0.8, bins=bins, label='1')
plt.xlabel('length')
plt.ylabel('numbers')
plt.legend(loc='upper right')
plt.xlim(0,500)
plt.grid()
plt.show()

## 4. Data cleaning

### acronym words

In [ ]:
acronym_words = []
acronym_words_dict = []
acronym_words = open('../input/mydata/acronym_word.txt','r', encoding='utf-8')
acronym_words = acronym_words.readlines()
for i in range(len(acronym_words)):
  acronym_words_split = acronym_words[i].split("\t")
  for j in range(len(acronym_words_split)):
    acronym_words_split[j] = re.sub("[\n\ufeff]",'',acronym_words_split[j])
  try:
    acronym_words_dict.append({acronym_words_split[0]:acronym_words_split[1]})
  except: 
    pass

acronym_words_dict[:5]


### Stop words

In [ ]:
stop_words = []
stop_words = open('../input/mydata/stop_word.txt','r', encoding='utf-8')
stop_words = stop_words.readlines()
for i in range(len(stop_words)):
  stop_words[i] = re.sub("[\t\n\ufeff]",'',stop_words[i])

stop_words[:5]

In [ ]:
"""**Tách từ**

"""

# Tách từ
def sementation(text):
  return ViTokenizer.tokenize(text)

In [ ]:
"""**Chuẩn hóa**"""

# Chuẩn hóa unicode sang chuẩn unicode dựng sẵn
def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic

dicchar = loaddicchar()
def covert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

In [ ]:
# Hàm chuẩn hoá câu
def standardize_data(row):
    # Xóa dấu chấm, phẩy, hỏi ở cuối câu
    row = re.sub(r"[\.,\?]+$-", "", row)
    # Xóa tất cả dấu chấm, phẩy, chấm phẩy, chấm thang, ... trong câu
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ")
    row = row.strip().lower()
    return row

In [ ]:
# Xử dụng bộ từ điển để thay thế các kiểu gõ dấu và viết tắt
def replace_acronyms(text):
  text_list = text.split(" ");
  for i in range(len(text_list)):
    for j in range(len(acronym_words_dict)):
      key = list(acronym_words_dict[j].keys())[0]
      value = list(acronym_words_dict[j].values())[0]
      if text_list[i] == key:
        text_list[i] = value
  return " ".join(text_list)


In [ ]:
# Xử lý các từ viết trùng lắp
def remove_loop_char(text):
  text = re.sub(r'([A-Z])\1+', lambda m: m.group(1).upper(), str(text), flags=re.IGNORECASE)
  text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',text)
  return text

In [ ]:
# Loại bỏ stop word
def remove_stop_word(text):
  split_word = text.split(" ");
  words = []
  for word in split_word:
    if word not in stop_words:
      words.append(word)
  return " ".join(words)

In [ ]:
"""**Tiền xử lý dữ liệu**"""
# Làm sạch dữ liệu
def text_prosessing(text):
    # Chuyển đổi thành chữ thường
    text = text.lower() 

    # tách từ
    text = sementation(text)

    #Chuẩn hóa unicode sang chuẩn unicode dựng sẵn
    text = covert_unicode(text)

    # Chuẩn hóa câu
    text = standardize_data(text)

    # Xử dụng bộ từ điển để thay thế các kiểu gõ dấu và viết tắt
    text = replace_acronyms(text)

    # Xử lý các từ viết trùng lắp
    text = remove_loop_char(text)

    # Xử dụng bộ từ điển để thay thế các kiểu gõ dấu và viết tắt
    text = remove_stop_word(text)

    # Xóa các kí tự trong dấu []
    text = re.sub('\[.*?\]', '', text)

    # Xóa các kí tự đặc biệt
    text = re.sub("\W",' ',text) 

    # Xóa các đường link
    text = re.sub('https?://\S+|www\.\S+', ' ', text)

    # Xóa các số
    text = re.sub('\w*\d\w*', '', text)

    # Xóa các đoạn mã html
    text = re.sub(r'<[*>]*>',' ', text)

    #Xóa các kí tự xuống dòng
    text = " ".join(re.sub("\n", " ", text).split())
    return text

In [ ]:
# Apply vào data
data['text'] = data['text'].apply(text_prosessing)

In [ ]:
data.head()

## 5. Build Model

In [ ]:
len(data[data['star'] < 4])

In [ ]:
len(data[data['star'] >= 4])

### Gộp 3 đánh giá 1 2 3 sao là đánh giá tiêu cực với nhãn là 0, 2 đánh giá 4 5 sao là tích cực với nhãn là 1

In [ ]:
data["star"] = data["star"].map({1:0, 2:0, 3:0, 4:1, 5:1})

### Our current data

In [ ]:
data.head(5)

In [ ]:
np.array(data['text'])[:3]

In [ ]:
X = data['text']
y = data['star']

In [ ]:
X.head(3)

In [ ]:
y.head()

### Sử dụng các giải thuật máy học để huấn luyện mô hình

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score

In [ ]:
cv = CountVectorizer()
X_vectors = cv.fit_transform(X)

In [ ]:
print(X_vectors.todense().shape)

In [ ]:
print(len(cv.get_feature_names()))
print(X_vectors.todense().shape)
print(X_vectors.todense()[0].shape)
print(len(X))

In [ ]:
# Huấn luyện mô hình bằng DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
total_accuracy = cross_val_score(model, X_vectors, y, cv=10, scoring='accuracy')
print('Accuracy:', np.mean(total_accuracy))

In [ ]:
# Huấn luyện mô hình bằng KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()
total_accuracy = cross_val_score(model, X_vectors, y, cv=10, scoring='accuracy')
print('Accuracy:', np.mean(total_accuracy))

In [ ]:
# Huấn luyện mô hình bằng MultinomialNB
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
total_accuracy = cross_val_score(model, X_vectors, y, cv=10, scoring='accuracy')
print('Accuracy:', np.mean(total_accuracy))

In [ ]:
# Huấn luyện mô hình bằng RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
total_accuracy = cross_val_score(model, X_vectors, y, cv=5, scoring='accuracy')
print('Accuracy:', np.mean(total_accuracy))

In [ ]:
# Huấn luyện mô hình bằng SVM
from sklearn.svm import SVC

model = SVC()
total_accuracy = cross_val_score(model, X_vectors, y, cv=5, scoring='accuracy')
print('Accuracy:', np.mean(total_accuracy))

### Thử cải thiện mô hình bằng PhoBERT

tham khảo: https://github.com/thangnch/MiAI_Sentiment_Analysis_PhoBert/blob/main/train_model.py

In [ ]:
!pip install underthesea

In [ ]:
from transformers import AutoModel, AutoTokenizer # Thư viện BERT
import torch
import underthesea

In [ ]:
# Hàm load model BERT
def load_bert():
    v_phobert = AutoModel.from_pretrained("vinai/phobert-base")
    v_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
    return v_phobert, v_tokenizer

In [ ]:
# Hàm tạo ra bert features
def make_bert_features(v_text):
    global phobert
    v_tokenized = []
    max_len = 100 # Mỗi câu dài tối đa 100 từ
    for i_text in v_text:
        print("Đang xử lý line = ", i_text)
        # Phân thành từng từ
        line = underthesea.word_tokenize(i_text)
        # Ghép lại thành câu như cũ sau khi lọc
        line = " ".join(line)
        line = underthesea.word_tokenize(line, format="text")
        print("Word segment  = ", line)
        # Tokenize bởi BERT
        line = tokenizer.encode(line)
        v_tokenized.append(line)

    # Chèn thêm số 1 vào cuối câu nếu như không đủ 100 từ
    padded = np.array([i + [1] * (max_len - len(i)) for i in v_tokenized])
#     print('padded:', padded[0])
#     print('len padded:', padded.shape)

    # Đánh dấu các từ thêm vào = 0 để không tính vào quá trình lấy features
    attention_mask = np.where(padded == 1, 0, 1)
#     print('attention mask:', attention_mask[0])

    # Chuyển thành tensor
    padded = torch.tensor(padded).to(torch.long)
#     print("Padd = ",padded.size())
    attention_mask = torch.tensor(attention_mask)

    # Lấy features dầu ra từ BERT
    with torch.no_grad():
        last_hidden_states = phobert(input_ids= padded, attention_mask=attention_mask)

    v_features = last_hidden_states[0][:, 0, :].numpy()
    print(v_features.shape)
    return v_features

In [ ]:
print("Chuẩn bị nạp model BERT....")
phobert, tokenizer = load_bert()
print("Đã nạp xong model BERT.")

In [ ]:
print("Chuẩn bị tạo features từ BERT.....")
global phobert
v_tokenized = []
max_len = 80 # Mỗi câu dài tối đa 90 từ
for i_text in X:
#     print("Đang xử lý line = ", i_text)
    # Phân thành từng từ
    line = underthesea.word_tokenize(i_text)
    # Ghép lại thành câu như cũ sau khi lọc
    line = " ".join(line)
    line = underthesea.word_tokenize(line, format="text")
#     print("Word segment  = ", line)
    # Tokenize bởi BERT
    line = tokenizer.encode(line)
    v_tokenized.append(line)

# Chèn thêm số 1 vào cuối câu nếu như không đủ 100 từ
padded = np.array([np.array(i[:max_len-len(i)]) if len(i) > max_len else np.array(i + [1] * (max_len - len(i))) for i in v_tokenized])
print('padded:', padded[0])
print('len padded:', padded.shape)


In [ ]:
# Đánh dấu các từ thêm vào = 0 để không tính vào quá trình lấy features
attention_mask = np.where(padded == 1, 0, 1)
print('attention mask:', attention_mask[0])

In [ ]:
# Đánh dấu các từ thêm vào = 0 để không tính vào quá trình lấy features
attention_mask = np.where(padded == 1, 0, 1)
print('attention mask:', attention_mask[0].shape)

In [ ]:
# Chuyển thành tensor
padded = torch.tensor(padded).to(torch.long)
print("Padd = ",padded.size())
attention_mask = torch.tensor(attention_mask)

In [ ]:
# Lấy features dầu ra từ BERT
with torch.no_grad():
    last_hidden_states = phobert(input_ids= padded, attention_mask=attention_mask)

v_features = last_hidden_states[0][:, 0, :].numpy()
print(v_features.shape)
print("Đã tạo xong features từ BERT")